# SUM vizualisation

In [2]:
%load_ext autoreload
%autoreload 2
import os
import sys
import panel as pn
import numpy as np
import pyvista as pv
import glob
from matplotlib.colors import ListedColormap
from omegaconf import OmegaConf

pn.extension('vtk')
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'
os.environ['PYVISTA_OFF_SCREEN'] = 'True'
os.environ['PYVISTA_USE_PANEL'] = 'True'

sh: 1: /usr/bin/Xvfb: not found


In [3]:
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.datasets.segmentation.sum import SUMDataset
from torch_points3d.datasets.segmentation import IGNORE_LABEL

## Load SUM dataset

In [4]:
DIR = os.path.dirname(os.getcwd())
dataset_options = OmegaConf.create()
dataset_options.data = OmegaConf.load(os.path.join(DIR,'conf/data/segmentation/sum.yaml'))

In [5]:
dataset_options.data.dataroot = os.path.join(DIR,"data")
dataset = SUMDataset(dataset_options.data)
print(dataset)

Dataset: SUMDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = Compose([
    GridSampling3D(grid_size=0.05, quantize_coords=False, mode=mean),
])
test_transform = Compose([
    RandomSphere(radius=20.0, center=False, sampling_strategy=SamplingStrategy(strategy=freq_class_based, class_weight_method=sqrt)),
    NormalizeRGB(True),
    AddFeatsByKeys(normal=True, rgb=True),
])
train_transform = Compose([
    RandomSphere(radius=20.0, center=False, sampling_strategy=SamplingStrategy(strategy=freq_class_based, class_weight_method=sqrt)),
    RandomNoise(sigma=0.001, clip=0.05),
    RandomRotate((-180, 180), axis=2),
    RandomScaleAnisotropic([0.8, 1.2]),
    Random symmetry of axes: x=True, y=True, z=False,
    NormalizeRGB(True),
    AddFeatsByKeys(normal=True, rgb=True),
])
val_transform = Compose([
    RandomSphere(radius=20.0, center=False, sampling_strategy=SamplingStrategy(strategy=freq_clas

## Visualise the data

In [6]:
OBJECT_COLOR = [
        [233, 229, 107],  #'ceiling' .-> .yellow
        [95, 156, 196],  #'floor' .-> . blue
        [179, 116, 81],  #'wall'  ->  brown
        [241, 149, 131],  #'beam'  ->  salmon
        [81, 163, 148],  #'column'  ->  bluegreen
        [77, 174, 84],  #'window'  ->  bright green
        [108, 135, 75],  #'door'   ->  dark green
        [41, 49, 101],  #'chair'  ->  darkblue
        [79, 79, 76],  #'table'  ->  dark grey
        [223, 52, 52],  #'bookcase'  ->  red
        [89, 47, 95],  #'sofa'  ->  purple
        [81, 109, 114],  #'board'   ->  grey
        [233, 233, 229],  #'clutter'  ->  light grey
        [0, 0, 0],  # unlabelled .->. black
    ]
def buil_cmap():
    mapping = np.linspace(0, len(OBJECT_COLOR), 256)
    newcolors = np.zeros((256, 3))
    for i, color in enumerate(OBJECT_COLOR):
        newcolors[mapping >= i-0.5] = np.asarray(color) / 255.
    return ListedColormap(newcolors)
cmap = buil_cmap()

In [7]:
def load_random_data(event):
    i = np.random.randint(0, len(dataset.train_dataset))
    sample = dataset.train_dataset[i]
    xyz = sample.pos.numpy()
    pl = pv.Plotter(notebook=True)
    point_cloud = pv.PolyData(xyz)
    point_cloud['label'] = sample.y.numpy()
    pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
    
    pan.object = pl.ren_win
    return point_cloud

In [8]:
pl = pv.Plotter(notebook=True)
pan = pn.panel(pl.ren_win, sizing_mode='scale_both', aspect_ratio=1,orientation_widget=True,)

In [9]:
button = pn.widgets.Button(name='Load new model', button_type='primary')
button.on_click(load_random_data)

In [10]:
dashboard = pn.Row(
    pn.Column('## S3DIS visualiser',button),
    pan
)

In [ ]:
dashboard

## Validation dataset

In [ ]:
i = 3
area = dataset.val_dataset[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()

## Train dataset

In [ ]:
i = 3
area = dataset.train_dataset._datas[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()

In [ ]:
i = 3
area = dataset.trainval_dataset._datas[i]
xyz = area.pos.numpy()
pl = pv.Plotter(notebook=True)
point_cloud = pv.PolyData(xyz)
point_cloud['label'] = area.y.numpy()
pl.add_points(point_cloud,cmap=cmap, clim=[0, len(OBJECT_COLOR)]) 
pl.show()